In [1]:
from multiprocessing import Pool, cpu_count
from skimage.color import rgb2hsv
from skimage.filters import gaussian
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import openslide as os
import pandas as pd
import numpy as np
import glob
import math
import cv2

from enum import Enum
from pathlib import Path
from collections import Counter
from skimage import color, filters, morphology
from PIL import Image, ImageDraw, ImageFilter
Image.MAX_IMAGE_PIXELS = None

# LOGGING #

In [2]:
import logging
import datetime
import sys


def prepare_logger(logger, level=logging.DEBUG):
    
    def prepare_handler(handler):
        handler.setLevel(level)
        handler.setFormatter(formatter)
        return handler

    formatter = logging.Formatter(fmt='%(asctime)s | [%(levelname)s] %(message)s',
                                  datefmt='%Y-%m-%d %H:%M:%S')

    del logger.handlers[:]
    logger.handlers.append(prepare_handler(logging.StreamHandler(sys.stderr)))

    logger.setLevel(level=level)
    return logger

logg = logging.getLogger()
logg = prepare_logger(logg)

# ANNOTATION UTILITY

In [3]:
def read_polygons(annotation_filepath, scale_factor, include_keywords, exclude_keywords):
    """
        Utility function to read an annotation XML file and create a list of vertices for
        polygon delimiting the cancerous area.
    """
    # if mark_tumor:
    #    exclude_keywords = ['Breast', 'Rough not-tumor']
    # else:
    #    exclude_keywords = ['Metastasis', 'Carcinoma', 'Tumor', 'Rough tumor']

    # Read cancer polygon area
    include_polygons = []
    exclude_polygons = []
    all_keywords = include_keywords + exclude_keywords

    root = ET.parse(annotation_filepath).getroot()
    for anno_tag in root.findall('Annotations/Annotation'):
        polygon = []

        keyword = anno_tag.get('PartOfGroup')

        if keyword in all_keywords:
            for coord in anno_tag.findall('Coordinates/Coordinate'):
                polygon.append((float(coord.get('X')) / scale_factor,
                                float(coord.get('Y')) / scale_factor))

        if keyword in include_keywords:
            include_polygons.append(polygon)
        elif keyword in exclude_keywords:
            exclude_polygons.append(polygon)

    return include_polygons, exclude_polygons

def get_annotation_mask(annot_file, include_keywords, exclude_keywords, size, scale_factor=1, bg_color='BLACK', poly_color='WHITE'):
    """
        Creates a binary mask for the cancer area (white) from annotation file
    """
    logg.info('Creating annotation mask.')
    mask = Image.new(
        'L', size=size, color=bg_color)
    draw = ImageDraw.Draw(mask)
    
    incl_polygons, excl_polygons = read_polygons(annot_file, scale_factor, include_keywords, exclude_keywords)
    
    for polygon in incl_polygons:
        draw.polygon(xy=polygon, outline=(poly_color), fill=(poly_color))
    
    for polygon in excl_polygons:
        draw.polygon(xy=polygon, outline=(bg_color), fill=(bg_color))
    
    return mask

# BACKGROUND MASKING

In [4]:
def get_bg_mask(slide, slide_name, base_dir, annot_xml_filepath, disk):

    # Get slide info
    _, _, bg_level, bg_scale_factor = get_slide_level_info(
        slide, magnification=10)

    # Create mask dir if does not exist
    masks_dir = base_dir / 'level{bg_level}/masks/'.format(bg_level=bg_level)
    create_dir(masks_dir)

    ### COMBINED BACKGROUND MASK PROCESS ###
    final_bg_dir = masks_dir / 'bg_final'
    final_bg_filepath = final_bg_dir / \
        '{slide_name}.png'.format(slide_name=slide_name)
    create_dir(final_bg_dir)

    if final_bg_filepath.exists():
        logg.debug('Combined mask already exists. Loading from disk.')
        combined_bg_mask = Image.open(str(final_bg_filepath))
        return combined_bg_mask

    ### BACKGROUND MASK PROCESS ###
    logg.debug('\'{}\' does not exists. Creating new mask.'.format(
        final_bg_filepath))
    bg_mask_dir = masks_dir / 'bg_init'
    bg_mask_filepath = bg_mask_dir / '{slide_name}-bg-level{bg_level}-hsv-otsu-disk{disk}-close-open.png'.format(
        slide_name=slide_name, bg_level=bg_level, disk=disk)
    create_dir(bg_mask_dir)

    if bg_mask_filepath.exists():
        logg.debug('Init background mask already exists. Loading from disk.')
        bg_mask = Image.open(str(bg_mask_filepath))
    else:
        bg_mask = create_bg_mask(slide, bg_level)
        bg_mask.save(str(bg_mask_filepath), format='PNG')

    ### ANNOTATION MASK PROCESS ###
    annot_dir = masks_dir / 'bg_annot'
    annot_fp = annot_dir / '{slide_name}.png'.format(slide_name=slide_name)
    create_dir(annot_dir)

    if annot_fp.exists():
        logg.debug('Exclusion mask already exists. Loading from disk.')
        annot_mask = Image.open(str(annot_fp))
    else:
        annot_mask = create_bg_annotation(
            annot_xml_filepath, size=slide.level_dimensions[bg_level], scale_factor=bg_scale_factor)
        annot_mask.save(str(annot_fp), format='PNG')

    # Combine masks
    combined_bg_mask = Image.fromarray(
        (np.array(bg_mask) & np.array(annot_mask)).astype(np.uint8) * 255, mode='L')
    combined_bg_mask.save(str(final_bg_filepath), format='PNG')
    return combined_bg_mask

def create_bg_annotation(annotation_fp, size, scale_factor):
    include_keywords = ['Rough not-tumor', 'Rough tumor', 'Tumor', 'Breast']
    exclude_keywords = []
    return get_annotation_mask(str(annotation_fp), include_keywords, exclude_keywords,
                        size=size, scale_factor=scale_factor,
                        bg_color='BLACK', poly_color='WHITE')

def create_bg_mask(slide, bg_level, disk=10):
    logg.debug('Retrieving scaled down slide image.')
    slide_img = slide.read_region(location=(
        0, 0), level=bg_level, size=slide.level_dimensions[bg_level]).convert('RGB')
    logg.debug('Converting image to HSV.')
    slide_hsv = color.rgb2hsv(slide_img)
    logg.debug('Saturating')
    saturation = slide_hsv[:, :, 1]
    logg.debug('Thresholding')
    threshold = filters.threshold_otsu(saturation)
    logg.debug('Thresholding saturation')
    high_saturation = (saturation > threshold)
    logg.debug('Creating disk')
    disk_object = morphology.disk(disk)
    logg.debug('Closing disk')
    mask = morphology.closing(high_saturation, disk_object)
    logg.debug('Opening disk')
    mask = morphology.opening(mask, disk_object)
    logg.debug('DONE')
    return Image.fromarray(mask)

# CANCER MASKING

In [5]:
def get_cancer_mask(slide, slide_name, base_dir, annot_xml_filepath, **kwargs):

    # Get slide info
    level, scale_factor, _, _ = get_slide_level_info(slide, magnification=10)

    # Create mask dir if does not exist
    cancer_mask_dir = base_dir / \
        'level{level}/masks/annotations'.format(level=level)
    cancer_mask_filepath = cancer_mask_dir / \
        '{slide_name}.png'.format(slide_name=slide_name)

    create_dir(cancer_mask_dir)

    if cancer_mask_filepath.exists():
        # Load mask from disk if already exists
        logg.debug('Cancer mask already exists. Loading from disk.')
        cancer_mask = Image.open(str(cancer_mask_filepath))
    else:
        # Create new annotation mask
        cancer_mask = create_cancer_annotation(annot_xml_filepath, size=slide.level_dimensions[level], scale_factor=scale_factor)
        cancer_mask.save(str(cancer_mask_filepath), format='PNG')

    return cancer_mask

def create_cancer_annotation(annotation_fp, size, scale_factor):
    include_keywords = ['Rough tumor', 'Tumor']
    exclude_keywords = []
    return get_annotation_mask(str(annotation_fp), include_keywords, exclude_keywords, size=size, scale_factor=scale_factor)

# UTILITY FUNCTIONS #

In [6]:
def tissue_percent(tile_mask, tile_size):
    """
        Calculates the fraction of non-black area vs total area in a mask
    """
    ts_count = np.count_nonzero(tile_mask)
    bg_count = tile_size*tile_size
    return ts_count / bg_count

def get_slide_level_info(slide, magnification, zero_magnification=20):

    if zero_magnification is None:
        logg.debug(
            'Zero magnification unspecificied. Reading from slide properties.')
        zero_magnification = int(slide.properties['openslide.objective-power'])

    level = slide.get_best_level_for_downsample(
        zero_magnification / magnification)
    scale_factor = slide.level_downsamples[level]
    logg.debug('Level {} scaling: {}'.format(level, scale_factor))

    bg_level = slide.get_best_level_for_downsample(zero_magnification / 1.25)
    bg_scale_factor = slide.level_downsamples[bg_level]

    logg.debug('BG Level {} scaling: {}'.format(bg_level, bg_scale_factor))
    logg.debug('Effective scaling: {}'.format(bg_scale_factor / scale_factor))
    return level, scale_factor, bg_level, bg_scale_factor

def calculate_progress(y_coord, x_coord, slide_width, slide_height, center_size, old_percentage):
    """
        Calculate progress as a ratio of processed slide area vs total slide area
    """
    total = slide_height*slide_width
    processed = (y_coord*slide_width + center_size*x_coord)
    percentage = int(processed / total * 100)

    if percentage > old_percentage:
        print('Progress: {:3}%\r'.format(percentage), end='')
    
    return percentage

def create_dir(dir_path):
    """ Creates a directory if does not exist already"""
    if not dir_path.exists():
        dir_path.mkdir(parents=True)

# MAIN LOGIC #

In [7]:
def main():
    TISSUE_TYPE = 'Mammy'
    DATA_DIR = Path('/home/matejg/Project/crc_ml/data/')
    SLIDE_DIR = DATA_DIR / 'raw' / TISSUE_TYPE / 'slides'
    XML_DIR = DATA_DIR / 'raw' / TISSUE_TYPE / 'annotations'

    disk = 10
    percent = -1
    background_threshold = 0.2
    cancer_threshold = 1/9
    tile_size = 96

    for slide_fn in SLIDE_DIR.glob('*.mrxs'):
        slide_name = slide_fn.stem
        logg.info('Processing: {}'.format(slide_name))

        # Skip if slide is not annotated
        annot_xml_filepath = (XML_DIR / slide_name).with_suffix('.xml')
        if not annot_xml_filepath.exists():
            logg.debug('XML Annotation does not exist. Skipping...')
            continue

        # Open WSI slide
        slide = os.open_slide(str(slide_fn))
        level, scaling_factor, bg_level, bg_scaling_factor = get_slide_level_info(slide, magnification=10)


        MAP_DIR = DATA_DIR / 'processed/{tissue_type}/level{tile_level}/tiles/'.format(tissue_type=TISSUE_TYPE, tile_level=level)
        BG_MASK_DIR = DATA_DIR / 'interim/{tissue_type}/'.format(tissue_type=TISSUE_TYPE)

        # Skip if only single level
        if slide.level_count < level+1:
            logg.debug('Not enough levels. Skipping...')
            continue

        # Retrieve masks
        bg_mask = get_bg_mask(slide, slide_name, BG_MASK_DIR, annot_xml_filepath, disk=disk)    
        cancer_mask = get_cancer_mask(slide, slide_name, BG_MASK_DIR, annot_xml_filepath)


        ### PROCESSING WSI ###
        logg.info('Processing WSI tiles.')
        slide_width, slide_height = slide.level_dimensions[level]
        effective_scaling_factor = bg_scaling_factor / scaling_factor

        offset_map = {'row': [], 'col': [], 'class': []}
        for row, y_coord in enumerate(range(0, slide_height, tile_size), 1):
            for col, x_coord in enumerate(range(0, slide_width, tile_size), 1):

                # CALCULATE PROGRESS
                percent = calculate_progress(
                    y_coord, x_coord, slide_width, slide_height, tile_size, percent)

                # RETRIEVE TILE FROM BACKGROUND MASK
                tile_mask = bg_mask.crop((int(x_coord // effective_scaling_factor), 
                                                  int(y_coord // effective_scaling_factor), 
                                                  int((x_coord+tile_size) // effective_scaling_factor), 
                                                  int((y_coord+tile_size) // effective_scaling_factor)))

                # SKIP IF TILE IS BACKGROUND
                if tissue_percent(np.array(tile_mask), int(tile_size // effective_scaling_factor)) < background_threshold:
                    continue

                # RETRIEVE CENTER REGION OF A TILE FROM CANCER MASK
                tile_mask = cancer_mask.crop((x_coord, y_coord, x_coord + tile_size, y_coord + tile_size))

                # TILE IS CANCER IFF 1/9 OF TILE IS CANCEROUS
                class_ = 0
                if tissue_percent(np.array(tile_mask), tile_size) >= cancer_threshold:
                    class_ = 1

                offset_map['row'].append(row)
                offset_map['col'].append(col)
                offset_map['class'].append(class_)

        offset_df = pd.DataFrame.from_dict(offset_map)
        offset_df.set_index(['col', 'row'], inplace=True)

        create_dir(MAP_DIR)

        output_fn = MAP_DIR / '{slide_name}-bg{bg_level}-tile{tile_size}.gz'.format(
            slide_name=slide_name,
            bg_level=bg_level, 
            tile_size=tile_size)
        offset_df.to_pickle(output_fn, compression='gzip')

# EXECUTOR #

In [8]:
if __name__ == '__main__':
    main()

2019-10-02 10:12:20 | [INFO] Processing: M023-2012_0597-03-C
2019-10-02 10:12:20 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:20 | [INFO] Processing: M028-2012_0702-03-C
2019-10-02 10:12:20 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:20 | [INFO] Processing: M009-2012_2901-14-R
2019-10-02 10:12:20 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:12:20 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:12:20 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:12:20 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:12:20 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:12:20 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:12:20 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:12:20 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:12:20 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:12:20 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:12:20 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:12:20 | [DEBUG] Effective scaling: 8.0
2019-10-02

2019-10-02 10:12:44 | [INFO] Processing: M026-2012_0698-03-C
2019-10-02 10:12:44 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:44 | [INFO] Processing: M005-2012_1326-01-C
2019-10-02 10:12:44 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:44 | [INFO] Processing: M006-2012_1365-01-C
2019-10-02 10:12:44 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:44 | [INFO] Processing: M008-2012_1404-03-C
2019-10-02 10:12:44 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:12:44 | [INFO] Processing: M025-2012_2969-02-R
2019-10-02 10:12:44 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:12:44 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:12:44 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:12:44 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:12:44 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:12:44 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:12:44 | [DEBUG] Combined mask already exists. Loading from disk.
2019-1

2019-10-02 10:13:08 | [INFO] Processing: M031-2012_0924-01-C
2019-10-02 10:13:08 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:13:08 | [INFO] Processing: M017-2012_4649-14-R
2019-10-02 10:13:08 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:08 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:08 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:08 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:08 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:08 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:08 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:13:08 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:13:08 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:13:08 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:08 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:08 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:08 | [DEBUG] Cancer mask already exists. Loading from disk.
2019-10-02 10:13:08 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:13

2019-10-02 10:13:29 | [INFO] Processing: M007-2012_1403-02-C
2019-10-02 10:13:29 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:13:29 | [INFO] Processing: M018-2012_1202-06-R
2019-10-02 10:13:29 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:30 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:30 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:30 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:30 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:30 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:30 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:13:30 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:13:30 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:13:30 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:30 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:30 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:30 | [DEBUG] Cancer mask already exists. Loading from disk.
2019-10-02 10:13:30 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:13

2019-10-02 10:13:51 | [INFO] Processing: M015-2012_0200-03-C
2019-10-02 10:13:51 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:13:51 | [INFO] Processing: M015-2012_4025-03-R
2019-10-02 10:13:51 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:13:51 | [INFO] Processing: M031-2012_2263-05-R
2019-10-02 10:13:51 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:51 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:51 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:51 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:51 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:51 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:13:51 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:13:51 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:13:51 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:13:51 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:13:51 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:13:51 | [DEBUG] Effective scaling: 8.0
2019-10-02

...

2019-10-02 10:22:08 | [INFO] Processing: M005-2012_1326-02-C
2019-10-02 10:22:08 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:08 | [INFO] Processing: M013-2012_0025-01-C
2019-10-02 10:22:08 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:08 | [INFO] Processing: M014-2012_0161-06-C
2019-10-02 10:22:08 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:08 | [INFO] Processing: M010-2012_2502-08-R
2019-10-02 10:22:08 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:08 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:08 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:08 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:08 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:08 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:08 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:22:08 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:22:08 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:22:08 | [DEBUG] Level

2019-10-02 10:22:31 | [INFO] Processing: M029-2012_0800-01-C
2019-10-02 10:22:31 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:31 | [INFO] Processing: M023-2012_1540-06-R


2019-10-02 10:22:32 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:32 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:32 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:32 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:32 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:32 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:32 | [DEBUG] Combined mask already exists. Loading from disk.
2019-10-02 10:22:32 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:22:32 | [DEBUG] STREAM b'IDAT' 41 65536
2019-10-02 10:22:32 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:32 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:32 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:32 | [DEBUG] Cancer mask already exists. Loading from disk.
2019-10-02 10:22:32 | [DEBUG] STREAM b'IHDR' 16 13
2019-10-02 10:22:32 | [DEBUG] STREAM b'IDAT' 41 210368
2019-10-02 10:22:32 | [INFO] Processing WSI tiles.


2019-10-02 10:22:55 | [INFO] Processing: M014-2012_0125-04-C
2019-10-02 10:22:55 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:55 | [INFO] Processing: M014-2012_0125-06-C
2019-10-02 10:22:55 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:55 | [INFO] Processing: M027-2012_0699-03-C
2019-10-02 10:22:55 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:55 | [INFO] Processing: M016-2012_0284-03-C
2019-10-02 10:22:55 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:55 | [INFO] Processing: M010-2012_1510-04-C
2019-10-02 10:22:55 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:22:55 | [INFO] Processing: M029-2012_1819-10-R
2019-10-02 10:22:55 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:55 | [DEBUG] BG Level 4 scaling: 16.0
2019-10-02 10:22:55 | [DEBUG] Effective scaling: 8.0
2019-10-02 10:22:55 | [DEBUG] Level 1 scaling: 2.0
2019-10-02 10:22:55 | [DEBUG] BG Level 4 scaling: 16.0
2019

2019-10-02 10:23:18 | [INFO] Processing: M030-2012_0804-01-C
2019-10-02 10:23:18 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:23:18 | [INFO] Processing: M007-2012_1403-01-C
2019-10-02 10:23:18 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:23:18 | [INFO] Processing: M020-2012_0434-02-C
2019-10-02 10:23:18 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:23:18 | [INFO] Processing: M016-2012_0284-02-C
2019-10-02 10:23:18 | [DEBUG] XML Annotation does not exist. Skipping...
2019-10-02 10:23:18 | [INFO] Processing: M015-2012_0200-02-C
2019-10-02 10:23:18 | [DEBUG] XML Annotation does not exist. Skipping...
